# Part 3 - Text analysis and ethics

# 3.a Computing PMI

In this assessment you are tasked to discover strong associations between concepts in Airbnb reviews. The starter code we provide in this notebook is for orientation only. The below imports are enough to implement a valid answer.

### Imports, data loading and helper functions

We first connect our google drive, import pandas, numpy and some useful nltk and collections modules, then load the dataframe and define a function for printing the current time, useful to log our progress in some of the tasks.

In [1]:
import pandas as pd
from nltk.tag import pos_tag
from nltk import RegexpParser
import re
from collections import defaultdict,Counter
from nltk.stem import WordNetLemmatizer
from datetime import datetime
from tqdm import tqdm
import numpy as np
import os
tqdm.pandas()

In [2]:
# nltk imports, note that these outputs may be different if you are using colab or local jupyter notebooks
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\c2086876\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\c2086876\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\c2086876\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# load stopwords
sw = set(stopwords.words('english'))

In [4]:
sw

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [5]:
'''We want to find the alternative forms of stopwords that have the "'" symbol in them 
in order to be able to add also to stopwords the word without this symbol'''

pattern = r'\w+\'\w+'

new_stopwords = []
for word in sw:
    # If it finds a word that contains "'" it appends the word in new_stopwords list
    if len(re.findall(pattern,word)) == 1:
        new_stopwords.append(re.findall(pattern,word)[0].replace('\'',''))
new_stopwords

['couldnt',
 'wouldnt',
 'neednt',
 'youre',
 'shes',
 'mustnt',
 'youd',
 'wasnt',
 'mightnt',
 'doesnt',
 'thatll',
 'dont',
 'its',
 'youll',
 'shouldnt',
 'shouldve',
 'havent',
 'didnt',
 'hasnt',
 'werent',
 'isnt',
 'arent',
 'shant',
 'youve',
 'wont',
 'hadnt']

In [6]:
# After checking those "new" words we add them to the stopwords variables named sw
for word in new_stopwords:
    sw.add(word)
sw

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'arent',
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'couldnt',
 'd',
 'did',
 'didn',
 "didn't",
 'didnt',
 'do',
 'does',
 'doesn',
 "doesn't",
 'doesnt',
 'doing',
 'don',
 "don't",
 'dont',
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'hadnt',
 'has',
 'hasn',
 "hasn't",
 'hasnt',
 'have',
 'haven',
 "haven't",
 'havent',
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'isnt',
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'mightnt',
 'more',
 'most',
 'mustn',
 "mustn't",
 'mustnt',
 'my',
 'myself',
 'needn',
 "needn't",
 'neednt',
 'no',
 'nor',
 'not',
 'no

In [7]:
basedir = os.getcwd()
df = pd.read_csv(os.path.join(basedir,'reviews.csv'))
# deal with empty reviews
df.comments = df.comments.fillna('')

In [8]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...


In [9]:
df.shape

(452143, 6)

### 3.a1 - Process reviews

What to implement: A `function process_reviews(df)` that will take as input the original dataframe and will return it with three additional columns: `tokenized`, `tagged` and `lower_tagged`.

In [10]:
def process_reviews(df):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
    # Initialize 3 lists one for each column we will create
    tokenized_col = []
    tagged_col = []
    lower_tagged_col = []


    mylen = len(df)
    count = 0
    
    # Iterate through the given dataframe
    for index, row in df.iterrows():
        # tokenize the words for the comments of a row
        token = word_tokenize(row.comments)
        # Append the tokenized words to the proper list
        tokenized_col.append(token)
        # Tag the tokenized words of the row and then append them to the proper list
        tagged_col.append(pos_tag(token))
        # lower_tagged.append(list(set(pos_tag([item.lower() for item in token]))))
        # Make the tagged words lowercased and then if they are not stopwords append them to the lower_tagged_col list
        lower_tagged_col.append(pos_tag([item.lower() for item in token if item.lower() not in sw]))
        count += 1

        if count % 1000 == 0:
            print(f'{count} out of {mylen}')

    # Set as values of the 3 new columns the proper list we created for each one
    df['tokenized'] = tokenized_col
    df['tagged'] = tagged_col
    df['lower_tagged'] = lower_tagged_col

    # Return the modified dataframe
    return df

In [11]:
# df = process_reviews(df)
df = process_reviews(df[:500])

### 3.a2 - Create a vocabulary

What to implement: A function `get_vocab(df)` which takes as input the DataFrame generated in step 1.c, and returns two lists, one for the 1,000 most frequent center words (nouns) and one for the 1,000 most frequent context words (either verbs or adjectives). 

In [12]:
def get_vocab(df):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
  cent_list, cont_list = [], []

  for review in df.lower_tagged:
    cent_list.extend([word for word in [list_of_words[0] for list_of_words in review if list_of_words[1][0] == 'N']])
    cont_list.extend([word for word in [list_of_words[0] for list_of_words in review if (list_of_words[1][0] == 'J') or (list_of_words[1][0] == 'V')]])
    
  cent_dict = Counter(cent_list)
  cont_dict = Counter(cont_list)

  cent_vocab = [key for key, value in sorted(cent_dict.items(), key=lambda item: item[1], reverse=True)][:1000]
  cont_vocab = [key for key, value in sorted(cont_dict.items(), key=lambda item: item[1], reverse=True)][:1000]

  return cent_vocab, cont_vocab

In [13]:
cent_vocab, cont_vocab = get_vocab(df)

In [14]:
cent_vocab[:5]

['daniel', 'room', 'place', 'host', 'location']

In [15]:
cont_vocab[:5]

['great', 'clean', 'alex', 'nice', 'comfortable']

In [16]:
samewords = [name for name in cent_vocab if name in cont_vocab]
len(samewords)

243

In [17]:
samewords

['daniel',
 'place',
 'location',
 'amsterdam',
 'stay',
 'apartment',
 'bathroom',
 'alex',
 'clean',
 'bus',
 'distance',
 'neighborhood',
 'maps',
 'bed',
 'experience',
 'get',
 'tram',
 'que',
 'walk',
 'airbnb',
 'transport',
 'thank',
 'bit',
 'visit',
 'très',
 'bien',
 'tips',
 'arrival',
 'muy',
 'helpful',
 'lot',
 'è',
 'places',
 'la',
 'cozy',
 'beds',
 'help',
 'perfect',
 'towels',
 'e',
 'tea',
 'daniels',
 'need',
 'die',
 'use',
 'convenient',
 'friend',
 'shower',
 'minute',
 'sehr',
 'check',
 'map',
 'tidy',
 'bike',
 'window',
 'pictures',
 'ist',
 'zu',
 'para',
 'stops',
 'nice',
 'es',
 'lo',
 'staying',
 'guest',
 'spotless',
 'garden',
 'wifi',
 'fun',
 'et',
 'séjour',
 'photos',
 '’',
 'por',
 'amazing',
 'stop',
 'work',
 'travel',
 'lots',
 'tourist',
 'le',
 'stairs',
 'cafes',
 'pleasant',
 'appartment',
 'beautiful',
 'luggage',
 'il',
 'ce',
 'drink',
 'welcoming',
 'feel',
 'restaurant',
 'enjoy',
 'dutch',
 'living',
 'recommend',
 'ride',
 'offer'

### 3.a3 Count co-occurrences between center and context words

What to implement: A function `get_coocs(df, center_vocab, context_vocab)` which takes as input the DataFrame generated in step 1, and the lists generated in step 2 and returns a dictionary of dictionaries, of the form in the example above. It is up to you how you define context (full review? per sentence? a sliding window of fixed size?), and how to deal with exceptional cases (center words occurring more than once, center and context words being part of your vocabulary because they are frequent both as a noun and as a verb, etc). Use comments in your code to justify your approach. 

In [18]:
# def get_coocs(df, cent_vocab, cont_vocab):
#   sentences = []
#   comments = df.comments

#   for comment in comments:
#     sentences.extend([sentence for sentence in comment.split('.')])
  
#   print('yolo')
#   # print(sentences)
  
#   coocs = {}
  
#   count = 0
#   for center_word in cent_vocab:
#     count += 1
#     words = []
#     for sentence in sentences:
#       if center_word in sentence:
#         words_in_sentence = word_tokenize(sentence)
#         words.extend([word for word in words_in_sentence if word in cont_vocab])
    
#     center_word_dict = dict(Counter(words))
#     coocs[center_word] = center_word_dict
#     print(f'{count} out of 1000')
    
#   # cent_dict = Counter(cent_list)
#   # cont_dict = Counter(cont_list)

  
#   return coocs  

In [19]:
def get_coocs(df, cent_vocab, cont_vocab):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.
        cent_vocab: The dataframe we want to modify.
        cont_vocab: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
    sentences = []
    comments = df.comments

    for comment in comments:
        sentences.extend([sentence for sentence in comment.split('.')])
  
    print('yolo')
  
    sentences_per_center_word = {center_word : Filter(sentences, center_word) for center_word in cent_vocab}

    print('swag')

    
    coocs = {}

    count = 0
    count2 = 0
    diff = 0
    for center_word, sentences in sentences_per_center_word.items():
        words = []
        count += 1
        start = pd.to_datetime('today')
        count2 = 0
        
        for sentence in sentences:
            count2 += 1
            words_of_sentence = [word for word in word_tokenize(sentence) if word in cont_vocab]
            if len(words_of_sentence) > 0: words.extend(words_of_sentence)
                
        end = pd.to_datetime('today')
        diff += (end-start).total_seconds()
        print(f'{center_word} - - - - {(end-start).total_seconds()} seconds')
        coocs[center_word] = dict(Counter(words))
        if count % 20 == 0 : print(f'{count} center_word out of {len(sentences_per_center_word)} in {diff/60} minutes')

            
    print(diff/60)
    return coocs 

def Filter(sentences, center_word):
    sentences_for_center_word = []
    
    for sentence in sentences:
        if center_word in sentence.split():
            sentences_for_center_word.append(sentence)
    print(f'{center_word} --- {len(sentences_for_center_word)}')
    return sentences_for_center_word

In [20]:
coocs = get_coocs(df, cent_vocab, cont_vocab)

97 seconds
recommande - - - - 0.002033 seconds
complaints - - - - 0.000959 seconds
facilities - - - - 0.002034 seconds
tutto - - - - 0.000992 seconds
assistance - - - - 0.000998 seconds
light - - - - 0.002962 seconds
temperature - - - - 0.002047 seconds
420 center_word out of 1000 in 0.0450908333333332 minutes
internet - - - - 0.001987 seconds
know - - - - 0.014987 seconds
shops - - - - 0.002011 seconds
okay - - - - 0.001987 seconds
name - - - - 0.000997 seconds
security - - - - 0.000999 seconds
hallway - - - - 0.0 seconds
rest - - - - 0.002956 seconds
er - - - - 0.002958 seconds
aufenthalt - - - - 0.0 seconds
gut - - - - 0.0 seconds
hätte - - - - 0.0 seconds
picture - - - - 0.000956 seconds
experiencia - - - - 0.001035 seconds
mejor - - - - 0.000994 seconds
cuida - - - - 0.000996 seconds
detalle - - - - 0.0 seconds
persona - - - - 0.002004 seconds
tipo - - - - 0.000991 seconds
gracias - - - - 0.002007 seconds
440 center_word out of 1000 in 0.04572191666666654 minutes
euro - - - - 0.00

### 3.a4 Convert co-occurrence dictionary to 1000x1000 dataframe
What to implement: A function called `cooc_dict2df(cooc_dict)`, which takes as input the dictionary of dictionaries generated in step 3 and returns a DataFrame where each row corresponds to one center word, and each column corresponds to one context word, and cells are their corresponding co-occurrence value. Some (x,y) pairs will never co-occur, you should have a 0 value for those cases. 

In [21]:
def cooc_dict2df(coocs):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        coocs: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
  coocdf = pd.DataFrame(columns=cont_vocab, index = cent_vocab)

  for index, row in coocdf.iterrows():
    for word in cont_vocab:
      try:
        coocdf[word][index] = coocs[index][word]
      except: 
        coocdf[word][index] = 0

  return coocdf

In [22]:
coocdf = cooc_dict2df(coocs)
coocdf.shape

(1000, 1000)

### 3.a5 Raw co-occurrences to PMI scores

What to implement: A function `cooc2pmi(df)` that takes as input the DataFrame generated in step 4, and returns a new DataFrame with the same rows and columns, but with PMI scores instead of raw co-occurrence counts. 

In [23]:
coocdf

,great,clean,alex,nice,comfortable,stay,recommend,good,daniel,gave,...,potential,cultivated,opt,published,explaning,extend,fifteen,centrum,kurzen,ausgefallen
daniel,1,1,0,0,0,3,2,0,8,0,...,0,0,0,0,0,0,0,0,0,0
room,26,53,0,26,46,11,2,13,0,3,...,0,1,0,0,0,0,1,1,0,0
place,28,39,0,11,17,29,24,14,0,7,...,0,0,0,0,0,0,0,0,0,0
host,23,6,0,11,3,12,3,3,1,6,...,0,0,0,0,0,0,0,0,0,0
location,28,9,0,9,9,12,4,8,0,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
würde,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jederzeit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fiancee,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
eateries,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
def cooc2pmi(df):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
  pmidf = pd.DataFrame(columns=cont_vocab, index = cent_vocab)

  N = 0
  for index, row in df.iterrows():
    N += sum(row)

  for index, row in df.iterrows():
    for word in cont_vocab:
      try:
        pmi = df[word][index] / (sum(df[word])/N / sum(row)/N)
        if pmi == 0:
          pmidf[word][index] = 0
        else:
          pmidf[word][index] = np.log([pmi])[0] 
#         print(pmidf[word][index])
      except: 
        pmidf[word][index] = 0
      
  return pmidf

In [25]:
pmidf = cooc2pmi(coocdf)
pmidf.shape

(1000, 1000)

In [26]:
pmidf

,great,clean,alex,nice,comfortable,stay,recommend,good,daniel,gave,...,potential,cultivated,opt,published,explaning,extend,fifteen,centrum,kurzen,ausgefallen
daniel,18.414304,18.365834,0,0,0,19.398869,19.918717,0,23.523611,0,...,0,0,0,0,0,0,0,0,0,0
room,24.814636,25.478360,0,25.288135,25.737073,23.840387,23.060952,24.768726,0,23.623089,...,0,27.071311,0,0,0,0,26.224013,25.972699,0,0
place,24.794201,25.077087,0,24.333391,24.647102,24.715245,25.451316,24.748291,0,24.375844,...,0,0,0,0,0,0,0,0,0,0
host,23.683307,22.291102,0,23.419208,21.998318,22.918672,22.457691,22.293663,23.577678,23.307510,...,0,0,0,0,0,0,0,0,0,0
location,24.276672,23.093222,0,23.615192,23.493585,23.315327,23.142028,23.671146,0,23.011017,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
würde,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jederzeit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fiancee,15.706254,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
eateries,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3.a6 Retrieve top-k context words, given a center word

What to implement: A function `topk(df, center_word, N=10)` that takes as input: (1) the DataFrame generated in step 5, (2) a `center_word` (a string like `‘towels’`), and (3) an optional named argument called `N` with default value of 10; and returns a list of `N` strings, in order of their PMI score with the `center_word`. You do not need to handle cases for which the word `center_word` is not found in `df`. 

In [33]:
# pmidf.iloc[2,:]
pmidf['place']['room']

23.846894894658952

In [38]:
def topk(df, center_word, N=10):
    '''
    This function takes as input the given dataframe and creates three new columns the tokenized, tagged and lower_tagged. 
    The tokenized column has as input the words of the comments for its row. The tagged has the result Part-of-speech (PoS) 
    tagging for the tokenized words and finally the lower_tagged column holds the tagged words in lowercase.

    Args:
        df: The dataframe we want to modify.
        center_word: The dataframe we want to modify.
        N: The dataframe we want to modify.

    Returns: A new version of the given dataframe with three additional columns: tokenized, tagged and lower_tagged.
    '''
    
  dicts_ = {word: df[word][center_word] for word in cont_vocab}
  top_words = [key for key, value in sorted(dicts_.items(), key=lambda item: item[1], reverse=True)][:N]

  return top_words

In [39]:
topk(pmidf, 'place')

['imagine',
 'rewarding',
 'rabble',
 'place',
 'worried',
 'calm',
 'impeccable',
 'finding',
 'choose',
 'satisfied']

In [40]:
topk(pmidf, 'location')

['walkable',
 'travelled',
 'location',
 'cool',
 'says',
 'takes',
 'numerous',
 'ideal',
 'slept',
 'dam']

In [41]:
topk(pmidf, 'coffee')

['tasty',
 'ant',
 'smelled',
 'tea',
 'sunny',
 'disposal',
 'benefit',
 'added',
 '/',
 'guidebooks']

# 3.b Ethical, social and legal implications



Local authorities in touristic hotspots like Amsterdam, NYC or Barcelona regulate the price of recreational apartments for rent to, among others, ensure that fair rent prices are kept for year-long residents. Consider your price recommender for hosts in Question 2c. Imagine that Airbnb recommends a new host to put the price of your flat at a price which is above the official regulations established by the local government. Upon inspection, you realize that the inflated price you have been recommended comes from many apartments in the area only being offered during an annual event which brings many tourists, and which causes prices to rise. 

In this context, critically reflect on the compliance of this recommender system with **one of the five actions** outlined in the **UK’s Data Ethics Framework**. You should prioritize the action that, in your opinion, is the weakest. Then, justify your choice by critically analyzing the three **key principles** outlined in the Framework, namely _transparency_, _accountability_ and _fairness_. Finally, you should propose and critically justify a solution that would improve the recommender system in at least one of these principles. You are strongly encouraged to follow a scholarly approach, e.g., with peer-reviewed references as support. 

Your report should be between 500 and 750 words long.  

### Your answer here. No Python, only Markdown.

Write your answer after the line.

---

...